In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
import csv
import json
import os
import numpy as np

In [ ]:
csv_file_path="/content/ai-medical-chatbot.csv"
json_file_path = "/content/data.json"

In [ ]:
if not(os.path.exists(json_file_path)):
  with open(json_file_path, 'w') as f:
    f.write('[]')

In [ ]:
with open(csv_file_path, 'r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    data=list(csv_reader)

In [ ]:
with open(json_file_path, 'w') as json_file:
    json.dump(data, json_file, indent=4)

In [ ]:
model=SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:

with open(json_file_path,'r') as f:
  data=json.load(f)

In [ ]:
import json
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

model = SentenceTransformer('all-MiniLM-L6-v2')

# Read your large JSON
with open('data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

batch_size = 8

# Open output file in write mode
with open('qa_embeddings.jsonl', 'w', encoding='utf-8') as out_file:
    for i in tqdm(range(0, len(data), batch_size)):
        batch = data[i:i + batch_size]
        patient_texts = [item["Patient"] for item in batch]
        doctor_answers = [item["Doctor"] for item in batch]

        try:
            batch_embeddings = model.encode(patient_texts)
        except Exception as e:
            print(f"Error encoding batch {i}: {e}")
            continue

        # Write each item individually to JSONL
        for text, answer, emb in zip(patient_texts, doctor_answers, batch_embeddings):
            record = {
                "Patient": text,
                "Doctor": answer,
                "Embedding": emb.tolist()
            }
            out_file.write(json.dumps(record) + "\n")


100%|██████████| 32115/32115 [10:28<00:00, 51.06it/s]


In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 18.1 MB/s eta 0:00:00


In [ ]:
import faiss
import numpy as np
import json

# Load all embeddings from the JSONL file
embeddings = []
answers = []

with open('/content/qa_embeddings.jsonl', 'r') as f:
    for line in f:
        item = json.loads(line)
        emb = np.array(item["Embedding"], dtype='float32')
        embeddings.append(emb)
        answers.append(item["Doctor"])

# Convert list to numpy array
embedding_matrix = np.vstack(embeddings)

# Build FAISS index
index = faiss.IndexFlatL2(embedding_matrix.shape[1])
index.add(embedding_matrix)

# Save index and answers
faiss.write_index(index, 'qa_index.faiss')
with open('qa_answers.json', 'w') as f:
    json.dump(answers, f)


In [ ]:
import faiss
import json
import numpy as np
from sentence_transformers import SentenceTransformer

# Load index and answers
index = faiss.read_index("qa_index.faiss")
with open("qa_answers.json", "r") as f:
    answers = json.load(f)

model = SentenceTransformer('all-MiniLM-L6-v2')

def fast_search(query, top_k=1):
    query_vector = model.encode([query]).astype('float32')
    D, I = index.search(query_vector, top_k)  # D = distances, I = indices
    results = [answers[i] for i in I[0]]
    return results

# Example
response = fast_search("What should I do for hair fall?")
print("Best Answer:", response[0])


Best Answer: Hello and Welcome to ‘Ask A Doctor’ service.I have reviewed your query and here is my advice.Firstly, it would help if you could narrow down on a cause for your hair fall. It is a common complaint and could be due to stress, work, poor diet, hair styling products, dandruff, change in water, etc., commonly.Also, you need to check for other causes like thyroid problems, low iron /hemoglobin, any illness like fever, typhoid, and any long-term medication. Any genetic hair loss, or balding in the family could make you more prone to hair fall. In females, hormonal changes, menopause, post pregnancy can cause hair fall. You can check with a dermatologist regarding any tests required to check for these causes.I would advise you to start on a hair supplement capsule like Keraglo or Follihair daily for 2 to 3 months to see improvement. It is just a hair supplement. Any treatment needs to be taken for a few months regularly to see good results. Apply almond oil at night but don't lea